In [3]:
import numpy as np
import math
import matplotlib.pyplot as plt  
from sklearn.metrics import mean_squared_error
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Bidirectional
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

import sys
import xlwt
from sklearn.preprocessing import MinMaxScaler
import datetime,pickle,os,glob

def transfromData(trainRaw, testRaw,windosSize):  ##Train ratial, train, test
    sc = MinMaxScaler(feature_range = (0, 1))

    npTrain = sc.fit_transform(np.array(trainRaw).reshape(-1,1))
    npTest = sc.fit_transform(np.array(testRaw).reshape(-1,1))
    
    X_train, y_train = splitXy(npTrain,windosSize)
    X_test, y_test = splitXy(npTest,windosSize)
    return sc, X_train, y_train, X_test, y_test

def splitXy(data,windosSize):
    windows = []
    for i in range(windosSize, data.shape[0]):
        windows.append(data[i-windosSize:i, 0])
    np.random.shuffle(windows)
    X = []
    y = []
    for i in range(len(windows)):
        X.append(windows[i][:6])
        y.append(windows[i][-(windosSize-6):])
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X,y
def buildModel(outSize):
    regressor = Sequential()
    #regressor.add(Bidirectional(LSTM(units=50,return_sequences=True),input_shape = (X_train.shape[1], 1)))
    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50,return_sequences=True))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units = outSize))
    # Compiling
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return regressor


def getStationList():
    with open('pickles/stationList.pickle', 'rb') as handle:
        stationList = pickle.load(handle)
    os.chdir('excelFiles/LSTM')
    replaceDict = ['.xls','LSTMresult']
    for direct in glob.glob("*.xls"):
        fileName = direct                                               
        for w in replaceDict:
            fileName = fileName.replace(w,'')
        
        if fileName in stationList:
            stationList.remove(fileName)
            print(fileName)
    os.chdir('../..')
    return stationList
def Visualize():
    predicted = sc.inverse_transform(regressor.predict(X_test))
    originY = sc.inverse_transform (y_test)
    print("MSE : ["+str(mean_squared_error(predicted, originY))+"]")
    # Visualising the results
    plt.plot(originY[:100], color = 'red', label = 'Real')  
    plt.plot(predicted[:100], color = 'blue', label = 'Predicted ') 
    plt.legend()
    plt.show()
def writeExcelHead(sheet1,epochs,station):
    sheet1.write(0,1,station)
    raw = 1
    for e in range(epochs):
        sheet1.write(raw,0,e+1)
        raw+=1

##get data##

def fetchData(station,windosSize):

    with open('pickles/'+station+'2017trainRaw.pickle', 'rb') as handle:
        trainRawData = pickle.load(handle)
    with open('pickles/'+station+'2017testRaw.pickle', 'rb') as handle:
        testRawData = pickle.load(handle)
        
    sc, X_train, y_train, X_test, y_test = transfromData(trainRawData,testRawData,windosSize)
    return sc, X_train, y_train, X_test, y_test

def train(regressor,sc, X_train, y_train, X_test, y_test,epochs):
    for i in range(epochs):
        regressor.fit(X_train, y_train,validation_split=0.2, epochs = 1, batch_size = 32,verbose=2)
        predicted = sc.inverse_transform(regressor.predict(X_test))
        originY = sc.inverse_transform (y_test)
        mse = mean_squared_error(predicted, originY)
        print("Epoch : " +str(i)+", MSE : ["+str(mse)+"]")
        print('-------------------------------------------')
  
    return mse

Using TensorFlow backend.


In [ ]:
epochs = 250
stationList = getStationList()
col=1
for station in stationList:
    print("training : " +station)
    MSEs = []
    for windowSize in range(7,31):
        sc, X_train, y_train, X_test, y_test = fetchData(station,windowSize)
        regressor = buildModel(windowSize-6)
        mse = train(regressor,sc,X_train, y_train, X_test, y_test,epochs)
        MSEs.append(mse)
    
    book = xlwt.Workbook(encoding="utf-8")
    sheet1 = book.add_sheet("Sheet1")
    writeExcelHead(sheet1,epochs,station)
    row = 1
    for m in MSEs:
        sheet1.write(row,1,m)
        row+=1
    book.save("excelFiles/LSTM/LSTMresult"+station+".xls")
        
    print('check point at ' + str(datetime.datetime.now()))







training : 二林
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 7s - loss: 0.0030 - val_loss: 0.0017
Epoch : 0, MSE : [49.107409056484045]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 0.0013 - val_loss: 8.9868e-04
Epoch : 1, MSE : [25.9824985957406]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 9.7529e-04 - val_loss: 9.1105e-04
Epoch : 2, MSE : [28.424164165657867]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 9.0147e-04 - val_loss: 8.9393e-04
Epoch : 3, MSE : [27.972422883580084]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 8.9306e-04 - val_loss: 8.3079e-04
Epoch : 4, MSE : [27.29163544113049]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1

 - 5s - loss: 7.7530e-04 - val_loss: 7.9261e-04
Epoch : 43, MSE : [24.890325370970555]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 7.5711e-04 - val_loss: 8.0257e-04
Epoch : 44, MSE : [25.07929858665016]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 7.6693e-04 - val_loss: 7.9185e-04
Epoch : 45, MSE : [25.249492308655462]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 7.6962e-04 - val_loss: 8.1975e-04
Epoch : 46, MSE : [25.861245915635457]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 6s - loss: 7.6424e-04 - val_loss: 7.7918e-04
Epoch : 47, MSE : [24.561977601070883]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 6s - loss: 7.8923e-04 - val_loss: 7.7919e-04
Epo

Epoch : 86, MSE : [26.200969927931308]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 7.5735e-04 - val_loss: 8.0252e-04
Epoch : 87, MSE : [25.24093220970427]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 6s - loss: 7.4584e-04 - val_loss: 8.0770e-04
Epoch : 88, MSE : [25.057628434289494]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 7.5983e-04 - val_loss: 7.8358e-04
Epoch : 89, MSE : [24.58179715975774]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 7.4836e-04 - val_loss: 7.9570e-04
Epoch : 90, MSE : [24.77506855812563]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 7.5572e-04 - val_loss: 7.8661e-04
Epoch : 91, MSE : [24.703975023047235]
--------------

 - 5s - loss: 7.5481e-04 - val_loss: 7.7175e-04
Epoch : 130, MSE : [25.090560850026296]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 7.3960e-04 - val_loss: 7.9518e-04
Epoch : 131, MSE : [24.642743267058492]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 7.2974e-04 - val_loss: 7.8172e-04
Epoch : 132, MSE : [24.95432648035222]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 7.3274e-04 - val_loss: 7.9453e-04
Epoch : 133, MSE : [24.9806451670594]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 7.4448e-04 - val_loss: 7.9045e-04
Epoch : 134, MSE : [24.335684325612096]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 7.3049e-04 - val_loss: 8.0821e-04


 - 5s - loss: 7.2280e-04 - val_loss: 7.5576e-04
Epoch : 173, MSE : [24.971558215034207]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 7.1811e-04 - val_loss: 7.4347e-04
Epoch : 174, MSE : [24.505879580610085]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 7.1844e-04 - val_loss: 8.0938e-04
Epoch : 175, MSE : [25.166259915319067]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 7.3307e-04 - val_loss: 7.6557e-04
Epoch : 176, MSE : [24.429295756584896]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 7.1245e-04 - val_loss: 7.6428e-04
Epoch : 177, MSE : [24.62405663275819]
-------------------------------------------
Train on 17140 samples, validate on 4285 samples
Epoch 1/1
 - 5s - loss: 7.2430e-04 - val_loss: 7.6408e-0

...

 - 7s - loss: 0.0043 - val_loss: 0.0045
Epoch : 202, MSE : [139.02861400080783]
-------------------------------------------
Train on 17125 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0043 - val_loss: 0.0045
Epoch : 203, MSE : [138.94826568009972]
-------------------------------------------
Train on 17125 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0043 - val_loss: 0.0045
Epoch : 204, MSE : [139.97585860731564]
-------------------------------------------
Train on 17125 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0043 - val_loss: 0.0045
Epoch : 205, MSE : [135.20221786208828]
-------------------------------------------
Train on 17125 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0043 - val_loss: 0.0045
Epoch : 206, MSE : [140.93497431148666]
-------------------------------------------
Train on 17125 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0043 - val_loss: 0.0045
Epoch : 207, MSE : [138.03923705113678]
-----

 - 7s - loss: 0.0043 - val_loss: 0.0045
Epoch : 247, MSE : [138.56515363348862]
-------------------------------------------
Train on 17125 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0043 - val_loss: 0.0045
Epoch : 248, MSE : [141.48360956381427]
-------------------------------------------
Train on 17125 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0043 - val_loss: 0.0045
Epoch : 249, MSE : [143.55408929947217]
-------------------------------------------
Train on 17124 samples, validate on 4282 samples
Epoch 1/1
 - 14s - loss: 0.0061 - val_loss: 0.0049
Epoch : 0, MSE : [158.89585263951346]
-------------------------------------------
Train on 17124 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0051 - val_loss: 0.0046
Epoch : 1, MSE : [142.1750655601521]
-------------------------------------------
Train on 17124 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0048 - val_loss: 0.0044
Epoch : 2, MSE : [135.67301866167801]
-----------

Epoch : 42, MSE : [140.42833300775308]
-------------------------------------------
Train on 17124 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0046 - val_loss: 0.0043
Epoch : 43, MSE : [136.59713789589745]
-------------------------------------------
Train on 17124 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0046 - val_loss: 0.0043
Epoch : 44, MSE : [138.4748460537903]
-------------------------------------------
Train on 17124 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0046 - val_loss: 0.0043
Epoch : 45, MSE : [133.39767879060108]
-------------------------------------------
Train on 17124 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0046 - val_loss: 0.0043
Epoch : 46, MSE : [135.3832782855656]
-------------------------------------------
Train on 17124 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0046 - val_loss: 0.0043
Epoch : 47, MSE : [133.24851991435102]
-------------------------------------------
Train on 

 - 7s - loss: 0.0045 - val_loss: 0.0043
Epoch : 88, MSE : [132.7868034404857]
-------------------------------------------
Train on 17124 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0045 - val_loss: 0.0043
Epoch : 89, MSE : [138.25573201532694]
-------------------------------------------
Train on 17124 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0045 - val_loss: 0.0043
Epoch : 90, MSE : [134.79543136486117]
-------------------------------------------
Train on 17124 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0045 - val_loss: 0.0043
Epoch : 91, MSE : [132.39163495046157]
-------------------------------------------
Train on 17124 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0045 - val_loss: 0.0043
Epoch : 92, MSE : [136.4885913432971]
-------------------------------------------
Train on 17124 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0045 - val_loss: 0.0043
Epoch : 93, MSE : [139.27793321909968]
-------------

 - 9s - loss: 0.0045 - val_loss: 0.0043
Epoch : 133, MSE : [138.74833892561884]
-------------------------------------------
Train on 17124 samples, validate on 4282 samples
Epoch 1/1
 - 8s - loss: 0.0045 - val_loss: 0.0043
Epoch : 134, MSE : [134.79911395883286]
-------------------------------------------
Train on 17124 samples, validate on 4282 samples
Epoch 1/1
 - 8s - loss: 0.0045 - val_loss: 0.0043
Epoch : 135, MSE : [134.6491052971076]
-------------------------------------------
Train on 17124 samples, validate on 4282 samples
Epoch 1/1
 - 7s - loss: 0.0045 - val_loss: 0.0043
Epoch : 136, MSE : [135.48743378172477]
-------------------------------------------
Train on 17124 samples, validate on 4282 samples
Epoch 1/1


In [11]:
with open('pickles/'+station+'2017trainRaw.pickle', 'rb') as handle:
    trainRawData = pickle.load(handle)

In [12]:
trainRawData

[37.0,
 40.0,
 42.0,
 40.0,
 36.0,
 39.0,
 40.0,
 43.0,
 39.0,
 44.0,
 46.0,
 50.0,
 46.0,
 38.0,
 45.0,
 43.0,
 59.0,
 58.0,
 54.0,
 44.0,
 40.0,
 46.0,
 47.0,
 41.0,
 43.0,
 45.0,
 46.0,
 42.0,
 41.0,
 51.0,
 56.0,
 52.0,
 43.0,
 39.0,
 35.0,
 36.0,
 37.0,
 41.0,
 35.0,
 29.0,
 24.0,
 28.0,
 27.0,
 35.0,
 35.0,
 34.0,
 29.0,
 25.0,
 28.0,
 28.0,
 29.0,
 28.0,
 20.0,
 16.0,
 12.0,
 17.0,
 20.0,
 22.0,
 18.0,
 20.0,
 21.0,
 23.0,
 19.0,
 20.0,
 17.0,
 19.0,
 13.0,
 18.0,
 18.0,
 23.0,
 24.0,
 26.0,
 12.0,
 17.0,
 20.0,
 19.0,
 20.0,
 21.0,
 19.0,
 11.0,
 9.0,
 14.0,
 16.0,
 21.0,
 24.0,
 24.0,
 20.0,
 18.0,
 25.0,
 21.0,
 24.0,
 28.0,
 36.0,
 32.0,
 28.0,
 25.0,
 26.0,
 22.0,
 22.0,
 17.0,
 18.0,
 15.0,
 20.0,
 23.0,
 25.0,
 34.0,
 40.0,
 31.0,
 27.0,
 17.0,
 24.0,
 21.0,
 29.0,
 29.0,
 29.0,
 27.0,
 26.0,
 16.0,
 16.0,
 17.0,
 32.0,
 28.0,
 20.0,
 17.0,
 20.0,
 22.0,
 22.0,
 20.0,
 20.0,
 24.0,
 33.0,
 31.0,
 30.0,
 21.0,
 25.0,
 26.0,
 27.0,
 27.0,
 25.0,
 26.0,
 29.0,
 27.0,
 24.0,
